In [ ]:
%pylab inline
import os
import keras
import numpy as np
import pandas as pd
import keras.backend as K

from time import time
from sklearn.cluster import KMeans
from keras import callbacks
from keras.models import Model, Sequential
from keras.optimizers import SGD
from keras.layers import Dense, Input, Lambda, Layer, Add, Multiply
from keras.initializers import VarianceScaling
from keras.engine.topology import Layer, InputSpec

from scipy.misc import imread
from sklearn.metrics import accuracy_score, normalized_mutual_info_score

Populating the interactive namespace from numpy and matplotlib


C:\Users\Tweety\AppData\Roaming\Python\Python37\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['imread', 'time']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [ ]:
import os 
import glob 
import numpy as np
import pandas as pd
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.utils import np_utils
import os
os.sys.path
import cv2
import matplotlib.pyplot as plt
import csv
from os.path import splitext
import PIL.Image
###### tr1 #######
file_dir = 'Untitled Folder'#./train_data/tr1/Untitled Folder
files = glob.glob(file_dir)
tr1 = []
for f1 in files:
    img_dir = os.path.join(f1,'*')
    images = glob.glob(img_dir)
    for img in images:
        img_temp = cv2.imread(img)
        img_temp=cv2.cvtColor(np.float32(img_temp),cv2.COLOR_BGR2GRAY)
        plt.axis('off')
#        plt.imshow(img_temp)
#        plt.show()
        #img_temp = cv2.resize(img_temp,(8250))
       # img_temp = img_temp.reshape(414,8250)
        tr1.append(img_temp) 
tr1 = np.array(tr1)
print("tr1.shape=",tr1.shape)

Using TensorFlow backend.


tr1.shape= (0,)


In [ ]:
train_x = tr1/255
train_x = train_x.reshape(-1, 396, 1080,1)
val_x = tr1/255.
val_x = val_x.reshape(-1, 396, 1080,1)

In [ ]:
# this is our input placeholder
input_img = Input(shape=(396, 1080,1))

# "encoded" is the encoded representation of the input
#encoded = Dense(500, activation='relu')(input_img)

#z_mu = Dense(10)(encoded)
#z_log_sigma = Dense(10)(encoded)
encoded = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
encoded = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
encoded = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)

z_mu = Dense(10)(encoded)
z_log_sigma = Dense(10)(encoded)
class KLDivergenceLayer(Layer):

    """ Identity transform layer that adds KL divergence
    to the final model loss.
    """

    def __init__(self, *args, **kwargs):
        self.is_placeholder = True
        super(KLDivergenceLayer, self).__init__(*args, **kwargs)

    def call(self, inputs):

        mu, log_sigma = inputs

        kl_batch = - .5 * K.sum(1 + log_sigma -
                                K.square(mu) -
                                K.exp(log_sigma), axis=-1)

        self.add_loss(K.mean(kl_batch), inputs=inputs)

        return inputs

z_mu, z_log_sigma = KLDivergenceLayer()([z_mu, z_log_sigma])
z_sigma = Lambda(lambda t: K.exp(.5*t))(z_log_sigma)

eps = Input(tensor=K.random_normal(shape=(K.shape(input_img)[0],10)))
z_eps = Multiply()([z_sigma, eps])
z = Add()([z_mu, z_eps])

decoder = Sequential([
    Dense(500, input_dim=10, activation='relu'),
    Dense(784, activation='sigmoid')
])

decoded = decoder(z)

# this model maps an input to its reconstruction
autoencoder = Model([input_img, eps], decoded)

NameError: name 'Input' is not defined

In [ ]:
autoencoder.summary()

Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_27 (InputLayer)           (None, 396, 1080, 1) 0                                            
__________________________________________________________________________________________________
conv2d_35 (Conv2D)              (None, 396, 1080, 16 160         input_27[0][0]                   
__________________________________________________________________________________________________
conv2d_36 (Conv2D)              (None, 396, 1080, 16 2320        conv2d_35[0][0]                  
__________________________________________________________________________________________________
conv2d_37 (Conv2D)              (None, 396, 1080, 16 2320        conv2d_36[0][0]                  
___________________________________________________________________________________________

In [ ]:
# this is our input placeholder
input_img = Input(shape=(396, 1080,1))

# "encoded" is the encoded representation of the input
encoded = Dense(500, activation='relu')(input_img)

z_mu = Dense(10)(encoded)
z_log_sigma = Dense(10)(encoded)

class KLDivergenceLayer(Layer):

    """ Identity transform layer that adds KL divergence
    to the final model loss.
    """

    def __init__(self, *args, **kwargs):
        self.is_placeholder = True
        super(KLDivergenceLayer, self).__init__(*args, **kwargs)

    def call(self, inputs):

        mu, log_sigma = inputs

        kl_batch = - .5 * K.sum(1 + log_sigma -
                                K.square(mu) -
                                K.exp(log_sigma), axis=-1)

        self.add_loss(K.mean(kl_batch), inputs=inputs)

        return inputs

z_mu, z_log_sigma = KLDivergenceLayer()([z_mu, z_log_sigma])
z_sigma = Lambda(lambda t: K.exp(.5*t))(z_log_sigma)

eps = Input(tensor=K.random_normal(shape=(K.shape(input_img)[0],10)))
z_eps = Multiply()([z_sigma, eps])
z = Add()([z_mu, z_eps])

decoder = Sequential([
    Dense(500, input_dim=10, activation='relu'),
    Dense(784, activation='sigmoid')
])

decoded = decoder(z)

# this model maps an input to its reconstruction
autoencoder = Model([input_img, eps], decoded)

In [ ]:
autoencoder.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           (None, 396, 1080, 1) 0                                            
__________________________________________________________________________________________________
dense_35 (Dense)                (None, 396, 1080, 50 1000        input_23[0][0]                   
__________________________________________________________________________________________________
dense_36 (Dense)                (None, 396, 1080, 10 5010        dense_35[0][0]                   
__________________________________________________________________________________________________
dense_37 (Dense)                (None, 396, 1080, 10 5010        dense_35[0][0]                   
___________________________________________________________________________________________

In [ ]:
def nll(y_true, y_pred):
    """ Negative log likelihood (Bernoulli). """

    # keras.losses.binary_crossentropy gives the mean
    # over the last axis. we require the sum
    return K.sum(K.binary_crossentropy(y_true, y_pred), axis=-1)

In [ ]:
autoencoder.compile(optimizer='adam', loss=nll)

In [ ]:
estop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')


In [ ]:
train_history = autoencoder.fit(train_x, train_x, epochs=50, batch_size=1, validation_data=(val_x, val_x), callbacks=[estop])

Train on 10 samples, validate on 5 samples
Epoch 1/50
10/10 [==============================] - 106s 11s/step - loss: 508.2521 - val_loss: 438.8027
Epoch 2/50
10/10 [==============================] - 101s 10s/step - loss: 392.6909 - val_loss: 321.9397
Epoch 3/50
10/10 [==============================] - 104s 10s/step - loss: 211.5648 - val_loss: 124.6898
Epoch 4/50
10/10 [==============================] - 113s 11s/step - loss: 108.8981 - val_loss: 83.6790
Epoch 5/50
10/10 [==============================] - 107s 11s/step - loss: 65.9320 - val_loss: 77.6402
Epoch 6/50
10/10 [==============================] - 114s 11s/step - loss: 60.5980 - val_loss: 46.2738
Epoch 7/50
10/10 [==============================] - 110s 11s/step - loss: 50.9745 - val_loss: 51.7841
Epoch 8/50
10/10 [==============================] - 141s 14s/step - loss: 49.2365 - val_loss: 45.3509
Epoch 9/50
10/10 [==============================] - 134s 13s/step - loss: 45.7147 - val_loss: 40.4280
Epoch 10/50
10/10 [=============

KeyboardInterrupt: 